In [ ]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [ ]:
import json

# 検索結果を返す関数の作成
def get_search_result(keyword, when="1", unit="m"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},
        ]
    }

    return json.dumps(result)

In [ ]:
from openai.types.chat import ChatCompletionToolParam

# ツール定義
tools=[
    ChatCompletionToolParam({
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "キーワード",
                    },
                    "when": {
                        "type": "number",
                        "description": "日付や時間の範囲"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"]
                    },
                },
                "required": ["keyword"],
            },
        },
    })
]

In [ ]:
# 言語モデルが直接回答できる質問
question = "日本の首都はどこにありますか？"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

# レスポンスの表示
pprint(vars(response))

In [ ]:
# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",
)

# レスポンスの表示
pprint(vars(response))

In [ ]:
# モデルがツール呼出と判断した
if response.choices[0].finish_reason == 'tool_calls':
    # 言語モデルの回答からツールを取得
    tool = response.choices[0].message.tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    print(f"関数名：{function_name}")

    # 引数の取得
    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    # 関数の実行
    function_response = globals()[function_name](**arguments)
    print("関数の実行結果：")
    pprint(json.loads(function_response))

    # 関数の実行結果をmessagesに加えて言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response_after_tool_call.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response.choices[0].message.content.strip())